In [1]:
import requests
import pandas as pd

In [27]:
def load_api_token(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

FUEL_API_TOKEN = load_api_token('fuel_api_token.txt')
TURSO_TOKEN = load_api_token('turso_token.txt')
TURSO_DB_URL = "libsql://cairns-fuel-h-unter.aws-ap-northeast-1.turso.io"
COUNTRY_ID = 21 # Australia
CAIRNS_REGION_ID = 16

In [20]:


FUEL_API_SUBSCRIBER_URL = "https://fppdirectapi-prod.fuelpricesqld.com.au/Subscriber"
FUEL_API_PRICE_URL = "https://fppdirectapi-prod.fuelpricesqld.com.au/Price"
headers = {
    "Authorization": f"FPDAPI SubscriberToken={FUEL_API_TOKEN}",
    "Content-Type": "application/json"
}

# Get Geographic Region

In [ ]:
# 1. Find Cairns GeoRegionId (dont need to run this every time as ive saved it to the constant CAIRNS_REGION_ID)
response_regions = requests.get(
    f"{FUEL_API_SUBSCRIBER_URL}/GetCountryGeographicRegions",
    headers=headers,
    params={"countryId": COUNTRY_ID}
)
geo_regions = response_regions.json()["GeographicRegions"]

cairns_region_id = next(
    region["GeoRegionId"]
    for region in geo_regions
    if region["Name"] == "Cairns" and region["GeoRegionLevel"] == 2
)

geo_regions_df = pd.DataFrame(geo_regions)
geo_regions_df
# print(f"Cairns GeoRegionId = {cairns_region_id}")
cairns_region_id

16

# Get Site Brand Details & Fuel Types

In [24]:
# these requests only need to be run if a new one is added that is unknown
response_brands = requests.get(
    f"{FUEL_API_SUBSCRIBER_URL}/GetCountryBrands",
    headers=headers,
    params={"countryId": COUNTRY_ID}
)
brands = response_brands.json()['Brands']
brands_df = pd.DataFrame(brands)
brands_df
brand_id_to_brand_name = {
    brand['BrandId']: brand['Name'] for brand in brands
}

resonse_fuel_types = requests.get(
    f"{FUEL_API_SUBSCRIBER_URL}/GetCountryFuelTypes",
    headers=headers,
    params={"countryId": COUNTRY_ID}
)
fuel_types = resonse_fuel_types.json()['Fuels']
fuel_types_df = pd.DataFrame(fuel_types)
fuel_types_df
fuel_type_id_to_fuel_name = {
    fuel['FuelId']: fuel['Name'] for fuel in fuel_types
}

In [25]:
fuel_types_df

,FuelId,Name
0,2,Unleaded
1,3,Diesel
2,4,LPG
3,5,Premium Unleaded 95
4,6,ULSD
5,8,Premium Unleaded 98
6,11,LRP
7,12,e10
8,13,Premium e5
9,14,Premium Diesel


# Get Cairns Site Details

In [26]:
# 3. Get site details for Cairns (site names, addresses, etc)
response_sites = requests.get(
    f"{FUEL_API_SUBSCRIBER_URL}/GetFullSiteDetails",
    headers=headers,
    params={
        "countryId": COUNTRY_ID,
        "geoRegionLevel": 2,
        "geoRegionId": cairns_region_id
    }
)
site_details = response_sites.json()["S"]
site_details_df = pd.DataFrame(site_details)
rename_map  = {
    "S": "site_id",
    "A": "address",
    "N": "site_name",
    "B": "brand_id",
    "P": "post_code",
    "Lat": "latitude",
    "Lng": "longitude"
}
site_details_df = site_details_df.rename(columns=rename_map)[list(rename_map.values())]
# site_details_df['brand_name'] = site_details_df['brand_id'].map(brand_id_to_brand_name)
site_details_df

,site_id,address,site_name,brand_id,post_code,latitude,longitude
0,61401973,68 Riverstone Road,United Gordonvale,23,4865,-17.099007,145.779894
1,61401991,3/167 Bruce Hwy,BP Edmonton,5,4869,-17.017429,145.743907
2,61402009,67 Alchera,Ampol Mossman,3421066,4873,-16.476177,145.374945
3,61402015,Captain Cook Hwy,Billabong Service Centre,12,4879,-16.771391,145.673257
4,61402073,131 Sheridan St,Ampol Foodary Cairns Sheridan St,3421066,4870,-16.918667,145.768858
...,...,...,...,...,...,...,...
56,61477987,237-241 Sheridan Street,Mobil Sheridan Street,16,4870,-16.911913,145.762875
57,61478023,69-70 Thomson Road,Marano’s Fuel Edmonton,12,4869,-17.019056,145.746202
58,61478205,334-336 Sheridan St,7-Eleven North Cairns,113,4870,-16.906147,145.759955
59,61478243,66-68 Anderson St,NightOwl BP Anderson Street,5,4870,-16.914498,145.748914


In [ ]:
# 2. Get all prices for Cairns
response_prices = requests.get(
    f"{FUEL_API_PRICE_URL}/GetSitesPrices",
    headers=headers,
    params={
        "countryId": COUNTRY_ID,
        "geoRegionLevel": 2,
        "geoRegionId": cairns_region_id
    }
)
site_prices = response_prices.json()["SitePrices"]
site_prices_df = pd.DataFrame(site_prices)
rename_map_prices = {
    "SiteId": "site_id",
    "FuelId": "fuel_type_id",
    "TransactionDateUtc": "transaction_date_utc",
    "Price": "price"
}
site_prices_df = site_prices_df.rename(columns=rename_map_prices)[list(rename_map_prices.values())]
# site_prices_df['fuel_name'] = site_prices_df['fuel_type_id'].map(fuel_type_id_to_fuel_name)
# site_prices_df['site_name'] = site_prices_df['site_id'].map(site_details_df.set_index('site_id')['site_name'])
site_prices_df

,site_id,fuel_type_id,transaction_date_utc,price
0,61401973,2,2025-09-04T04:55:04.663,1749.0
1,61401991,2,2025-10-03T04:29:56.417,1749.0
2,61402009,2,2025-08-14T00:07:09.38,1789.0
3,61402015,2,2025-08-14T00:32:18.32,1759.0
4,61402073,2,2025-10-29T02:40:00,1739.0
...,...,...,...,...
213,61477011,14,2025-10-20T23:15:24.787,1785.0
214,61477020,14,2025-10-28T14:01:00,1909.0
215,61477837,14,2025-10-29T03:26:36.273,1939.0
216,61478205,14,2025-10-29T00:57:00,1799.0


In [13]:
# plot the locations of the fuel stations on a map
import folium
cairns_map = folium.Map(location=[-16.9186, 145.7781], zoom_start=12)

# Add markers for each fuel station
for site_id, site_data in site_code_to_site_data.items():
    folium.Marker(
        location=[site_data["latitude"], site_data["longitude"]],
        popup=site_data["name"],
    ).add_to(cairns_map)

cairns_map.save("cairns_fuel_stations_map.html")
# show the map
cairns_map


In [33]:
print(merged.head())

    site_id  fuel_type_id     transaction_date_utc   price fuel_name  \
0  61401973             2  2025-09-04T04:55:04.663  1749.0  Unleaded   
1  61401991             2  2025-10-03T04:29:56.417  1749.0  Unleaded   
2  61402009             2   2025-08-14T00:07:09.38  1789.0  Unleaded   
3  61402015             2   2025-08-14T00:32:18.32  1759.0  Unleaded   
4  61402073             2      2025-10-26T14:01:00  1799.0  Unleaded   

                        site_name_x   latitude   longitude  \
0                 United Gordonvale -17.099007  145.779894   
1                       BP Edmonton -17.017429  145.743907   
2                     Ampol Mossman -16.476177  145.374945   
3          Billabong Service Centre -16.771391  145.673257   
4  Ampol Foodary Cairns Sheridan St -16.918667  145.768858   

                        site_name_y  
0                 United Gordonvale  
1                       BP Edmonton  
2                     Ampol Mossman  
3          Billabong Service Centre  
4  A

In [35]:
import folium
from folium import Element
import pandas as pd

# 1) Prep
site_prices_df['price'] = pd.to_numeric(site_prices_df['price'], errors='coerce')

# If you already merged, reuse it; otherwise merge now (keeps both names)
# merged = site_prices_df.merge(site_details_df[['site_id','latitude','longitude','site_name']],
#                               on='site_id', how='left', suffixes=('_price','_site'))

# 2) Coalesce site name into a single column
if 'site_name_final' not in merged.columns:
    name_cols = [c for c in ['site_name_y','site_name_x','site_name'] if c in merged.columns]
    merged['site_name_final'] = None
    for c in name_cols:
        merged['site_name_final'] = merged['site_name_final'].fillna(merged[c])

# 3) Keep rows with coords + price
merged_clean = merged.dropna(subset=['latitude','longitude','price']).copy()

# Optional: one marker per site & fuel (cheapest only)
merged_markers = (merged_clean
                  .sort_values('price', ascending=True)
                  .drop_duplicates(['site_id','fuel_name'], keep='first'))

# 4) Top-5 cheapest overall
top5 = merged_clean.sort_values('price', ascending=True).head(5)
top5_items = []
for _, r in top5.iterrows():
    price_cl = f"{r['price']/10:.1f} c/L"
    item = f"<li><strong>{r['site_name_final']}</strong> — {price_cl} <em>({r['fuel_name']})</em></li>"
    top5_items.append(item)
top5_html_list = "<ol style='margin:6px 0 0 18px; padding:0;'>" + "\n".join(top5_items) + "</ol>"

# 5) Map + markers
m = folium.Map(location=[-16.9186, 145.7781], zoom_start=12)

for _, row in merged_markers.iterrows():
    popup = (
        f"<b>{row['site_name_final']}</b><br>"
        f"{row['fuel_name']}<br>"
        f"Price: {row['price']/10:.1f} c/L<br>"
        f"Recorded (UTC): {row.get('transaction_date_utc','')}"
    )
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup,
        tooltip=row['site_name_final']
    ).add_to(m)

# 6) Floating panel (top-right)
panel_html = f"""
<div id="top5-panel" style="
  position: fixed; top: 10px; right: 10px; z-index: 9999;
  background: rgba(255,255,255,0.95); border: 1px solid #ccc; border-radius: 6px;
  padding: 10px 12px; font-family: Arial, sans-serif; font-size: 14px; max-width: 320px;
  box-shadow: 0 2px 8px rgba(0,0,0,0.2);">
  <div style="font-weight: 600; margin-bottom: 6px;">Top 5 Cheapest (All Fuels)</div>
  {top5_html_list}
  <div style="margin-top: 6px; font-size: 12px; color: #666;">
    Prices shown as c/L. Data from latest snapshot.
  </div>
</div>
"""
m.get_root().html.add_child(Element(panel_html))

m.save("cairns_fuel_stations_map.html")
m
